In [1]:
import json, random, re, collections, itertools, base64, sys,pickle
from dataclasses import dataclass
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(str(x)))
from pprint import pprint as pp
from edit_distance import edit_distance
ls = lambda path: list(map(str, path.iterdir()))

sys.path.append('../src')
annot_path = Path("../annotations")
data_path = Path("../data")
preprocessed_path = Path("../preprocessed")
mturk_path = Path("../mturk/results")

def read_annotation(annotation_id, mturk_batch=None):
    if mturk_batch:
        annotation = annotation_io.mturk_annotation(mturk_batch, annotation_id)
    else:
        annotation = annotation_io.get_annotation(annotation_id)
    return annotation

import annotation_io
import read_data
from instruction_parsing import program
annotation_io.set_recipe_source("npn-cooking")

%load_ext autoreload
%autoreload 2

In [2]:
mturk_batch = 4335716

In [3]:
with (mturk_path / f"{mturk_batch}.json").open('r') as f:
    annotations = json.load(f)

In [4]:
assignments = sum(annotations.values(),[])

In [5]:
train_set = dict()
for annotation_id in tqdm(assignments):
    try:
        annot = annotation_io.mturk_annotation(mturk_batch, annotation_id)
    except:
        print ("Cannot read")
        continue
    try:
        prog = program(annot)
    except:
        print ("Cannot simulate")
        continue
    train_set[annotation_id] = {"program":prog,
                                "instructions": annot["instructions"],
                                "ingredients": annot["normalized_ingredients"],
                                "title": annot["title"],
                                "status": annot["status"],
                                "id": annot["id"],
                               }

Cannot read
Cannot read
Cannot read



In [6]:
with (preprocessed_path / f"trainset_{mturk_batch}.json").open('wb') as f:
    pickle.dump(train_set,f)